 # extract Word2Vec embeddings for hashtags

In [ ]:
from cleantext import clean
from tqdm import tqdm
from nltk.tokenize import TweetTokenizer
import preprocessor as p 
import hdbscan
tqdm.pandas(desc="my bar!")
from cleantext import clean
from tqdm import tqdm
tqdm.pandas(desc="my bar!")
import preprocessor as p 
from nltk import bigrams
import networkx as nx
import pandas as pd
import re
from gensim.models import Word2Vec
from time import time
from matplotlib import rcParams
rcParams['figure.figsize'] = 20,20
import matplotlib.pyplot as plt
import umap
import hdbscan

def remove_words(string,rex):
    words_list = string.split(",")
    new_list = []
    for word in words_list:
        if rex.search(word) is None:
            # Didn't find a match
            new_list.append(word)

    new_string = ",".join(new_list)
    new_string=new_string.replace(' ','')

    return new_string.upper()
def add_hash(_lst):
    return ['#'+x for x in _lst if len(_lst)>1]
def hashtagsprocess(zdf0):
    badwords=[#'qanon',
        'deepstate','corona virus','coronavirus',
        'covid19','2019-nCoV','SARS-CoV-2','wuhanpneumonia']+['covid','corona','virus']
    rex_string = "(" + "|".join(badwords) + ")"
    rex = re.compile(rex_string, re.IGNORECASE)
    
    zdf0['hashtags']=zdf0['hashtags'].apply(lambda x : x.strip('[]'))
    zdf0['hashtags']=zdf0['hashtags'].apply(lambda x : x.replace("'",""))
    zdf0['hashtags']=zdf0['hashtags'].apply(lambda x: remove_words(x,rex))
    zdf0['hashtags']=zdf0['hashtags'].str.split(',')
    zdf0['hashtags']=zdf0['hashtags'].apply(lambda x: add_hash(x))
    return zdf0

def prepro(ndf,col):
    
    import re
    def remove_rt(query):
        query=re.sub(r"\brt", "", query)
        return query
    ndf[col]=ndf[col].progress_apply(lambda x: remove_rt(x))

    def tokenizer(_text):
        return TweetTokenizer().tokenize(_text)

    ndf[col]=ndf[col].progress_apply(lambda x:tokenizer(x))

    from nltk.corpus import stopwords
    STOPWORDS=stopwords.words('english')
    def remove_stop(_text):
        return ' '.join([x for x in _text if x not in STOPWORDS])

    ndf[col]=ndf[col].progress_apply(lambda x: remove_stop(x))
    
    def preprocessing(string): 

        text=clean(string,lower=True, no_emails=True,no_numbers=True,no_punct=True,no_digits=False,no_currency_symbols=True,
                   replace_with_number="",
                   no_urls=True,replace_with_url="",
                   replace_with_email="",
                   replace_with_currency_symbol="")

        return text
    ndf[col]=ndf[col].progress_apply(lambda x: preprocessing(x))
    
    def remove_emoji(string):
        p.set_options(p.OPT.EMOJI, p.OPT.SMILEY)
        
        return p.clean(string)
    ndf[col]=ndf[col].progress_apply(lambda x: remove_emoji(x))
    
    ndf.reset_index(inplace=True,drop=True)
    return ndf

def bihashtagGraph(_wdf0,_i):
    terms_bigram = [list(bigrams(tweet)) for tweet in _wdf0['hashtags']]

    d=[x for x in terms_bigram if x!=[]]

    lst=[]
    for each in d:
        if len(each)==0:
            lst.append(each)
        else:
            [lst.append(x) for x in each]

    hadf=pd.DataFrame(lst)

    K=nx.from_pandas_edgelist(hadf,0,1)

    nx.write_gexf(K,'../Project3/rebuttle_letter/'+str(_i)+'.gexf')
    return K

gf=pd.read_csv('../Project3/gf.csv')
df=gf[['text','hashtags']]

df0=hashtagsprocess(df)
df0=prepro(df0,'text')

df0['hashtags']=df0['hashtags'].apply(lambda x: ' '.join(x))
df0['hashtags']=df0['hashtags'].apply(lambda x: x.lower())
df0['whole']=df0['text']+' '+df0['hashtags']
df0.drop_duplicates(subset=['whole'],inplace=True)

w2v_model = Word2Vec(min_count=1,
                     window=5,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=11)

t = time()
lst= df0['whole'].tolist()
sentences=[x.split() for x in lst]
#[for each in x for x in sentences if each!='<not>'|'|']
w2v_model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

w2v_model.wv.most_similar(positive=["#qanon"])

df=pd.read_csv('../Project3/gf.csv')
df=df[['created_at','hashtags']]
df['created_at']=pd.to_datetime(df['created_at']).apply(lambda x: x.strftime('%Y/%m/%d'))
df['created_at']=pd.to_datetime(df['created_at'])
wdf=df[df['hashtags']!='[]']
wdf.sort_values(['created_at'],inplace=True)
wdf0=hashtagsprocess(wdf)
wdf0['created_at']=pd.to_datetime(wdf0['created_at']).apply(lambda x: x.strftime('%Y/%m'))

graphs=[]
topics=[]
degree=[]
topn=30
data=pd.DataFrame()
for i in [2,3,4,5,6,7]:
    ndf=wdf0[wdf0['created_at']=='2020/0'+str(i)]
    graph=bihashtagGraph(ndf,i)
    graphs.append(graph)
    lst=sorted(graph.degree(graph.nodes), key=lambda x: x[1], reverse=True)
    top10=[x[0] for x in lst[:topn]]
    topdegree=[x[1] for x in lst[:topn]]
    topics.append(top10)
    degree.append(topdegree)
    tmdf=pd.DataFrame([top10,topdegree]).T
    tmdf.columns=['topics','degree']
    tmdf['date']='2020/0'+str(i)
    data=pd.concat([data,tmdf])
    


data0=data.pivot("topics", "date", "degree")
data0=data0.fillna(0)

#ax = sns.heatmap(data0, xticklabels=True,cmap="YlGnBu",cbar_kws={'label': 'degree'})
#plt.tight_layout()
#plt.xticks(rotation=45)
#label_size = 100
#plt.rcParams['xtick.labelsize'] = label_size 
#plt.savefig('../Project3/rebuttle_letter/hashtags_heatmap'+str(topn)+'.pdf')

qlst=list(data0.index)
qlowerlst=[x.lower() for x in qlst]
[w2v_model.n_similarity(['#qanon'],[each]) for each in qlowerlst]

ndf=df0[df0['hashtags']!='']

ndf['hashtags']=ndf['hashtags'].str.split()

ndf0=ndf[['hashtags']]

ndf1=ndf0.explode('hashtags')

ndf1.drop_duplicates(inplace=True)
hashtag_lst=ndf1['hashtags'].tolist()

# Plot

In [ ]:
#HDBSCAN is used for computing the clustering word2Vec embeddings and UMAP reduces the dimension of those embeddings. 
umap_args = {
    "n_neighbors": 15,
    "n_components": 2, # 5 -> 2 for plotting 
    "metric": "cosine",
}
embeddings=w2v_model[hashtag_lst]
umap_model = umap.UMAP(**umap_args).fit(embeddings)

#embeddings=w2v_model.wv.vectors
print(embeddings.shape)
umap_embeddings=umap_model.fit_transform(embeddings)
print(umap_embeddings.shape)

# 1. HDBSCAN
cluster = hdbscan.HDBSCAN(min_cluster_size=15,
                          metric='euclidean',                      
                          cluster_selection_method='eom').fit(umap_embeddings)
# 2. UMAP
umap_data = umap.UMAP(n_neighbors=15, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
result = pd.DataFrame(umap_data, columns=['x', 'y'])

result['labels'] = cluster.labels_

fig, ax = plt.subplots(figsize=(20, 10))
outliers = result.loc[result.labels == -1, :]
clustered = result.loc[result.labels != -1, :]
plt.scatter(outliers.x, outliers.y, color='#BDBDBD', s=0.05)
plt.scatter(clustered.x, clustered.y, c=clustered.labels, s=0.05, cmap='hsv_r')
plt.colorbar()